In [2]:
import pandas as pd
import random

# Load the manifest file
file_path = 'GDC Manifest Aug 22.txt'

# Read the file into a DataFrame
df = pd.read_csv(file_path, sep='\t')

# Randomly select 1000 rows
df_sampled = df.sample(n=1000, random_state=42)

# Save the sampled DataFrame to a new file
output_path = 'manifest_sampled.txt'
df_sampled.to_csv(output_path, sep='\t', index=False)

output_path


'manifest_sampled.txt'